In [4]:
import pandas as pd
import numpy as np

In [6]:
#train_embeddings from bert embedding, dk why i stored in such a weird way lol but i blame gpt
train_embeddings = np.load('./dataset/train_bert_embeddings.npy')
valid_embeddings = np.load('./dataset/valid_bert_embeddings.npy')
test_embeddings = np.load('./dataset/test_bert_embeddings.npy')
train_labels = pd.read_csv("./dataset/train_with_bert_embeddings.csv")
valid_labels = pd.read_csv("./dataset/valid_with_bert_embeddings.csv")
test_labels = pd.read_csv("./dataset/test_with_bert_embeddings.csv")

In [8]:

# Ensure indices are integers
train_indices = train_labels['embedding_index'].values.astype(int)
valid_indices = valid_labels['embedding_index'].values.astype(int)
test_indices = test_labels['embedding_index'].values.astype(int)

# Extract embeddings using indices
train_X = train_embeddings[train_indices]
valid_X = valid_embeddings[valid_indices]
test_X = test_embeddings[test_indices]

# Extract labels (assuming 'label' column contains integer class labels)
train_Y = train_labels['label'].values.astype(int)
valid_Y = valid_labels['label'].values.astype(int)
test_Y = test_labels['label'].values.astype(int)

# Print shapes to verify
print(f"train_X shape: {train_X.shape}, train_Y shape: {train_Y.shape}")
print(f"valid_X shape: {valid_X.shape}, valid_Y shape: {valid_Y.shape}")
print(f"test_X shape: {test_X.shape}, test_Y shape: {test_Y.shape}")


train_X shape: (133999, 768), train_Y shape: (133999,)
valid_X shape: (17223, 768), valid_Y shape: (17223,)
test_X shape: (17063, 768), test_Y shape: (17063,)


In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Model Architecture Hyperparameters
input_shape = train_X.shape[1]  # Number of features in embeddings
dense_1_units = 128  # Neurons in first dense layer
dense_1_activation = "relu"  
dropout_1_rate = 0.3  # Dropout rate for first dropout layer
dense_2_units = 64  # Neurons in second dense layer
dense_2_activation = "relu"  
dropout_2_rate = 0.3  # Dropout rate for second dropout layer
output_units = len(np.unique(train_Y))  # Number of classes in output layer
output_activation = "softmax"  # Activation for classification

# Training Hyperparameters
optimizer = "adam"  # Optimizer choice
learning_rate = 0.001  # Learning rate
loss_function = "sparse_categorical_crossentropy"  # Loss function for classification
metrics = ["accuracy"]  # Metrics to track

# Training Parameters
batch_size = 32  # Batch size for training
epochs = 15  # Number of epochs to train
validation_data = (valid_X, valid_Y)  # Validation data

# Model Compilation & Training
model = keras.Sequential([
    layers.Input(shape=(input_shape,)),
    layers.Dense(dense_1_units, activation=dense_1_activation),
    layers.Dropout(dropout_1_rate),
    layers.Dense(dense_2_units, activation=dense_2_activation),
    layers.Dropout(dropout_2_rate),
    layers.Dense(output_units, activation=output_activation)
])

model.compile(optimizer=optimizer,
              loss=loss_function,
              metrics=metrics)

history = model.fit(train_X, train_Y, validation_data=validation_data,
                    epochs=epochs, batch_size=batch_size)

Epoch 1/15
4188/4188 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - accuracy: 0.3675 - loss: 1.4150 - val_accuracy: 0.4290 - val_loss: 1.3182
Epoch 2/15
4188/4188 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.4153 - loss: 1.3394 - val_accuracy: 0.4371 - val_loss: 1.3105
Epoch 3/15
4188/4188 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.4238 - loss: 1.3233 - val_accuracy: 0.4356 - val_loss: 1.3147
Epoch 4/15
4188/4188 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.4258 - loss: 1.3165 - val_accuracy: 0.4379 - val_loss: 1.3014
Epoch 5/15
4188/4188 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.4318 - loss: 1.3094 - val_accuracy: 0.4352 - val_loss: 1.3004
Epoch 6/15
4188/4188 ━━━━━━━━━━━━━━━━━━━━ 4s 989us/step - accuracy: 0.4381 - loss: 1.3024 - val_accuracy: 0.4443 - val_loss: 1.2928
Epoch 7/15
4188/4188 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.4379 - loss: 1.2980 - val_accuracy: 0.4407 - val_loss: 1.2969
Epoch 8/15
4188/4188 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.4410 - loss: 1.2951 

In [13]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_X, test_Y)
print(f"Test Accuracy: {test_acc:.4f}")

# Predict on the test set
test_predictions = model.predict(test_X)
test_predicted_classes = np.argmax(test_predictions, axis=1)

534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 406us/step - accuracy: 0.4965 - loss: 1.2222
Test Accuracy: 0.4448
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 302us/step
